# Parallel Retrieval

In **Retrieval-Augmented Generation (RAG)**, relevant documents are fetched from a knowledge base before the large language model (LLM) generates a response.
Typically, a single retriever is used, such as **FAISS, Pinecone, or Chroma.**

However, certain applications require querying multiple data sources—such as vector stores, APIs, or indices—and performing these queries sequentially can be slow.
This challenge is addressed through **parallel retrieval**, where multiple sources are queried simultaneously to improve efficiency and reduce latency.

## Use case

**Imagine a product assistant that retrieves info from:**

- Product Descriptions (stored in FAISS)

- Protection Plans (stored in Chroma)

- Customer Reviews (stored in Elasticsearch)

<br>
FAISS: 200ms

Chroma: 250ms

Elasticsearch: 180ms

**Total sequential:** 630ms

<br>

**With parallel retrieval, they run concurrently:**

Total time ≈ 250ms  ✅


In [3]:
import asyncio
import nest_asyncio
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [4]:
# Product knowledge base
products = [
    "iPhone 15 Pro has a titanium body and A17 chip.",
    "Samsung Galaxy S24 Ultra has 200MP camera.",
    "Google Pixel 8 has AI-powered photo editing."
]

# Warranty knowledge base
warranties = [
    "Warranty covers accidental and liquid damage.",
    "AppleCare+ extends iPhone coverage for 2 years.",
    "Samsung Care+ includes screen replacement twice per year."
]

# Reviews knowledge base
reviews = [
    "I love my iPhone 15 Pro. The titanium finish feels premium.",
    "Pixel 8 AI photo tools are amazing but battery life is average.",
    "Samsung S24 Ultra camera is the best I've ever used."
]


### Parallel Retrieval

In [5]:
nest_asyncio.apply()

# Initialize embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create vector stores for each dataset
product_store = FAISS.from_texts(products, embedding=embeddings)
warranty_store = FAISS.from_texts(warranties, embedding=embeddings)
review_store = FAISS.from_texts(reviews, embedding=embeddings)

# Define async retrieval functions
async def retrieve_from_product(query):
    return product_store.similarity_search(query, k=2)

async def retrieve_from_warranty(query):
    return warranty_store.similarity_search(query, k=2)

async def retrieve_from_reviews(query):
    return review_store.similarity_search(query, k=2)

# Combine in parallel
async def parallel_retrieve(query):
    results = await asyncio.gather(
        retrieve_from_product(query),
        retrieve_from_warranty(query),
        retrieve_from_reviews(query)
    )
    # Flatten the results
    flat_results = [item for sublist in results for item in sublist]
    return flat_results

# Run it
query = "Does iPhone have screen damage coverage?"
results = asyncio.run(parallel_retrieve(query))

print("🔍 Retrieved documents:")
for r in results:
    print("-", r.page_content)

🔍 Retrieved documents:
- iPhone 15 Pro has a titanium body and A17 chip.
- Google Pixel 8 has AI-powered photo editing.
- AppleCare+ extends iPhone coverage for 2 years.
- Warranty covers accidental and liquid damage.
- I love my iPhone 15 Pro. The titanium finish feels premium.
- Pixel 8 AI photo tools are amazing but battery life is average.
